<a href="https://colab.research.google.com/github/mhtasnia/panda_practice/blob/main/Exercise_Indexing%2C_Selecting_%26_Assigning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2051b0ee0f32cd1ed0318cdfc1ad7ac953f339cfba638d0fdd8f651a93c1809ff955c72bba93eb617f70a76785b04bd7cc530f353b17956c1fd4a4edf56a2d40868dad3ab727ff7fe5567b564316eb3ab694b569a5cea7592cd325a607842426baa288c054badc9a7a23449f3dc4ffc4b7569f974ba200470ca2bdff61d5bcb6b59ec831cd400e29e863a0f4680e13356563ae4512c80b2978d4cc2499c4b7ab4a7d677b69b47f54b17787db4259159ee5e3d1f9ec38d3f0d95100fc2ecb856f779a2e98022a243d4e226e92bdf21f9f1431377fa23eadc1c6eb18ff8894a573c83fc5eb35bf7d2962e9f1764b9f4a2499b71701a94a2699f48dab603c05d29d,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D985832e98e4cd1a27bf9738f2900c5c5acad4339135b461c9694fefe3e21c878213591cb402b439f2fb071ff8e7435ddfb40fdbdf0e0888f5b979d928e98b443e34fbeee8070ee12bcbe16d4693bbc70425e0cf6d74cfc08337e21c2c1968a3c922f845667693dd7b9049d7b06a7946c2a2a9503a3d327736f6a5911fdd2f10314e73cc8240a1f4cc595819d5d0872effeb7d1e6bc30b7c9a07af211d65904e566d9dd539617f4ede2e455bc2ec4b483dfc20867755a6cb1484186f695d003d0bb22662cdbfd82b51a965e063443a470af1178b72693e06ae762feabf0137b2b774a855363d5b85da36be9911d6b8e12f1ff3045355dc7b8ce84924345643725,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4dfd2f88693ba1b13a515bc416119d1999a277b3d3986b558bd35e4c1f54d0291292ef3dd0a21a4b4424e29877ac0ea8678633efdcbc501330dbbd4f743fb7da85a2dbb9499656a72aaddbdbdd0847a220e6134c224780a79eed8c492bfad73c466b41cf604cf78a908b3f7201380aa9e8f34d9dd99aa3706fe83f5caeb3f7fb98c9abafbf9bd542f2d24f8df19cecad7d1a9719b7949b939dec034c53e6fa476d2a10a3f705f66ba1611484dcb66292ab8d7fb385927bcb25678b5e706f7701683e2f505160e58c309ab3e1e3b5cc6e80ddc6c00a9f392dc0707fd9533012697256c1d27c603bd22a83a43f4312dc39d9465624879ac79bf7a1284ebb2a0102,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D03d5a9b48a3f84963564fe0426dd68990ed7121731b42206d45f2be4d9f89b1eb9a8207cbc3ff0e8f67656d0898671ad855848eda103c0932d72a25e1ba8f7a1f9d2be60c4ff39278d7a2b01534257056f624b4b6efa83ddcfe659f255e3494de23c7bce321b670a17d7758310c7b52245753149841d446f42b40cd891546525767d1cd9b49ce62320e90b686ef335ebcfb845017eafbbc24107b854f742fe889b29c096ae340e14e3d6099021d77e74385336306151166cd3e488d8841a55e910a32fab9f72095c853cda18a4daf9b13ee786f0463d9261f27f3673b7d499023b8dee4e52b7831903d9df470978d3bdbecc03a350ddfef086ddcf8780526c43,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D448539f49d1ccb3e047996f6b00290dda6155448ac317db09a5cb3149abceb3f475e112a3622cbd86a0d3545c1286c66fb20aeabb13bf95909b2a6e8c87d054aff59fb42147bbfdd5483254cd05b2b7c116bd13a165dc4e3c46ee5662d2dec2e9a7c292264c332db48ced68287cec2b45746dfca056743ef259889ee7bfb52200a5a96946d5b6d3fa946abcbde271c83db61d17e9721eaf59a0633bc8f6c61b74bf5f2e7bb4636cb42039b6930c685aab2ee3d80baad313be86ef065701bf43491389cc302ba8d9e6d0aef1f094f5e3f02a3a67123f96434240fdb269523539e64a890a4104670bd70416a96afbfc27a51e8467f177b614c7a161a4010364335,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38c88a680b439fcf531b784c47cf7cc283ae2057dd2fb52019216798309a76cd478e80746861f38afe5e118796d29857f6f569db6c2a31288129a0200973019e74dc59120d6c7fbee3bb01a9b676526a782895e00d01d66d11e8008c7ba65a4e960dfd3b2cc637fe19db17a3bc2c69147b8583c0aba6c4f18d3c2e22d0fc62d1cfab417666eb9833182a69415048ddad119dc7f5eabec2afdbcb8771af933ee1aa90b763838b1006043376819c12c0270c06ca666884b367789e7159d5e8ff808d0a821a1aedd09b5fbbd0ff5aa76728a6858606ebbccbd594a56cd8f22460e2b84b13bb1ec6ba1e45ec02da2df7c3a856a6129a92da900930a92e6fb0c693d6,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D128b86066981cb5fd20de0a6d85f5958cbc22e5d6dd868fd604e509dcaa51e944791bc33a805f738243b44ac0580210f4d778029f2382534ec751df84ad5e880de08b0dc42ca3ea3b3d1823e75fcd4cf8fecbc0259caf947850cad159c2f53ad11278ae9996cfe35a9a241bb27a62f97a8a005d78f82aa5547b931898daae646754a944b38922d490116f1b0974e9b99240e476cb3d0258b1d1ea969e97eb5fd0b7164d4d87e459d6615b3471c89c600507ae0fae8892d893a53f72e7d1f65fa6ce685f46ce31a2469d4ef88c4fc3fb5f779cfac9bf9464f5b4833c26c4b2cf2c40d444c2c1e9e5b130b868b960719594665b46230d5deff151becc7b5269d00,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093751Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1649b2dec90a0c902fa73573824c51ddf9587f3d331a91e3dd08ec2ba6f0712969fb43fffdcb0e51114288c2c288d81a6fa09c2024440abd64ae7f625fb39bb568f8c102c772001fcadf05bc608b54028f4445cbeb5470b850677402e4bf5e4242c9e068d072cd7aae373f64cb60dcea3f1a9fd11e5d77548f56a33d6bbc8acd1f08494cdc1d9f206675436ad691822addadfc82214c1f51227fa03c5fc03ca5d5ac4358f36a268ac7b6787e659a790c45263fd33d78f10647df786a22b660fb41afb1a9677aa599a60b7895aae72292692d20ccf09acd07f967e6bff20838f173181de3a956a9e174c5f3ae472a6ddb3e27cb710cb46036e116a3751bdba97a,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093752Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D681bddbb2815d79584c509251f0bdac494665eae29a8f523fb3fc1cfac6767a3d2b4d598c0e4836a43f5766f626bc929b5760c99334116d1fb65fac0bb58e3eb92c5b9c024efb76497da69045850cf1faad645a929112fa0085ff24219679a78ff929f52e03ba47fe875af626160375982a50aedf9de229fe700349ff1295871654d745c65f3753a1421a38bd413f8dfe7f298b51bfaacf5fd38c3ae5d9908a3bd391c3ce40b20bd4b1cf4faabb62beee28a71e9337ac97047d3ba140521b7bd47bce45e3c135d9005f6be3dcd28d6a2044b6f6e025cf3bbd4a8b11e127c1c985b03474df56658832af28d9d6f4d8df934e10a12d4e558cd4ae26f4fa37d3791,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T093752Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D92c866a5ade7c04b4b3cd30a498cfda02db988609f7a0926bb8d77ee8c83e2852af9a5007b3c5286dc1c88b30fba64b10bff56469b93eaa4569b525f9831876cf505af5e08d5b785908e2f7f352e7bc52b61c7acfb90667c10bd40b2709ea41741ba6ad481154d88e8b8fa0b0d43c6d553a6e677fe71713fed72531aaf5ccc5401337f56b4b96a28dd86c21c3aa193c8ef0519533863033c2b1171ea7ec9f877f826fc07e1f5d24b1a86fa363a13bb5d775dfa0ec88d26db883a8aeb96a7995487840e91c4a73a1ba3ee32fa265b22e71169774ce3a62396e4c13561fd31e046683718b3185b28e1537d3cbcdd5a680b8d4e335e6152a406693cd298147e969c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/indexing-selecting-assigning).**

---


# Introduction

In this set of exercises we will work with the [Wine Reviews dataset](https://www.kaggle.com/zynicide/wine-reviews).

Run the following cell to load your data and some utility functions (including code to check your answers).

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.indexing_selecting_and_assigning import *
print("Setup complete.")

Setup complete.


Look at an overview of your data by running the following line.

In [ ]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Exercises

## 1.

Select the `description` column from `reviews` and assign the result to the variable `desc`.

In [ ]:
# Your code here
desc = reviews.description

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Follow-up question: what type of object is `desc`? If you're not sure, you can check by calling Python's `type` function: `type(desc)`.

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Select the first value from the description column of `reviews`, assigning it to variable `first_description`.

In [ ]:
first_description = reviews.description.iloc[0]

# Check your answer
q2.check()
first_description

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_description = reviews.description.iloc[0]
```
Note that while this is the preferred way to obtain the entry in the DataFrame, many other options will return a valid result, such as `reviews.description.loc[0]`, `reviews.description[0]`, and more!  


"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [ ]:
#q2.hint()
#q2.solution()

## 3.

Select the first row of data (the first record) from `reviews`, assigning it to the variable `first_row`.

In [ ]:
first_row = reviews.iloc[0]

# Check your answer
q3.check()
first_row

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

country                                                    Italy
description    Aromas include tropical fruit, broom, brimston...
                                     ...                        
variety                                              White Blend
winery                                                   Nicosia
Name: 0, Length: 13, dtype: object

In [ ]:
#q3.hint()
q3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
first_row = reviews.iloc[0]
```

## 4.

Select the first 10 values from the `description` column in `reviews`, assigning the result to variable `first_descriptions`.

Hint: format your output as a pandas Series.

In [ ]:
first_descriptions = reviews.description.iloc[0:10]

# Check your answer
q4.check()
first_descriptions

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_descriptions = reviews.description.iloc[:10]
```
Note that many other options will return a valid result, such as `desc.head(10)` and `reviews.loc[:9, "description"]`.    


0    Aromas include tropical fruit, broom, brimston...
1    This is ripe and fruity, a wine that is smooth...
                           ...                        
8    Savory dried thyme notes accent sunnier flavor...
9    This has great depth of flavor with its fresh ...
Name: description, Length: 10, dtype: object

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Select the records with index labels `1`, `2`, `3`, `5`, and `8`, assigning the result to the variable `sample_reviews`.

In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/sHZvI1O.png)

In [ ]:
# i = [1, 2, 3, 5, 8]
sample_reviews = reviews.loc[[1,2,3,5,8]]

# Check your answer
q5.check()
sample_reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel


In [ ]:
#q5.hint()
q5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
indices = [1, 2, 3, 5, 8]
sample_reviews = reviews.loc[indices]
```

## 6.

Create a variable `df` containing the `country`, `province`, `region_1`, and `region_2` columns of the records with the index labels `0`, `1`, `10`, and `100`. In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/FUCGiKP.png)

In [ ]:
df = reviews.loc[[0, 1, 10, 100], ['country', 'province', 'region_1', 'region_2']]

# Check your answer
q6.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,province,region_1,region_2
0,Italy,Sicily & Sardinia,Etna,NaN
1,Portugal,Douro,NaN,NaN
10,US,California,Napa Valley,Napa
100,US,New York,Finger Lakes,Finger Lakes


In [ ]:
#q6.hint()
#q6.solution()

## 7.

Create a variable `df` containing the `country` and `variety` columns of the first 100 records.

Hint: you may use `loc` or `iloc`. When working on the answer this question and the several of the ones that follow, keep the following "gotcha" described in the tutorial:

> `iloc` uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded.
`loc`, meanwhile, indexes inclusively.

> This is particularly confusing when the DataFrame index is a simple numerical list, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return 1000 entries, while `df.loc[0:1000]` return 1001 of them! To get 1000 elements using `loc`, you will need to go one lower and ask for `df.iloc[0:999]`.

In [ ]:
df = reviews.loc[:99, ['country', 'variety']]
# Correct:

# cols = ['country', 'variety']
# df = reviews.loc[:99, cols]

# Check your answer
q7.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
cols = ['country', 'variety']
df = reviews.loc[:99, cols]
```
or 
```python
cols_idx = [0, 11]
df = reviews.iloc[:100, cols_idx]
```


,country,variety
0,Italy,White Blend
1,Portugal,Portuguese Red
...,...,...
98,Italy,Sangiovese
99,US,Bordeaux-style Red Blend


In [ ]:
#q7.hint()
#q7.solution()

## 8.

Create a DataFrame `italian_wines` containing reviews of wines made in `Italy`. Hint: `reviews.country` equals what?

In [ ]:
italian_wines = reviews[reviews.country == 'Italy']

# Check your answer
q8.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q8.hint()
q8.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
italian_wines = reviews[reviews.country == 'Italy']
```

## 9.

Create a DataFrame `top_oceania_wines` containing all reviews with at least 95 points (out of 100) for wines from Australia or New Zealand.

In [ ]:
top_oceania_wines = reviews.loc[(reviews.country.isin(['Australia', 'New Zealand']))& (reviews.points>=95)]

# Check your answer
q9.check()
top_oceania_wines

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
345,Australia,This wine contains some material over 100 year...,Rare,100,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards
346,Australia,"This deep brown wine smells like a damp, mossy...",Rare,98,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscadelle...,Muscadelle,Chambers Rosewood Vineyards
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122507,New Zealand,"This blend of Cabernet Sauvignon (62.5%), Merl...",SQM Gimblett Gravels Cabernets/Merlot,95,79.0,Hawke's Bay,NaN,NaN,Joe Czerwinski,@JoeCz,Squawking Magpie 2014 SQM Gimblett Gravels Cab...,Bordeaux-style Red Blend,Squawking Magpie
122939,Australia,Full-bodied and plush yet vibrant and imbued w...,The Factor,98,125.0,South Australia,Barossa Valley,NaN,Joe Czerwinski,@JoeCz,Torbreck 2013 The Factor Shiraz (Barossa Valley),Shiraz,Torbreck


In [ ]:
#q9.hint()
q9.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
top_oceania_wines = reviews.loc[
    (reviews.country.isin(['Australia', 'New Zealand']))
    & (reviews.points >= 95)
]
```

# Keep going

Move on to learn about **[summary functions and maps](https://www.kaggle.com/residentmario/summary-functions-and-maps)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*